In [6]:
import threading
import time

import numpy as np
import torch

import mmcv
from mmdet3d.apis import init_model
from mmdet3d.core.bbox import get_box_type
import matplotlib.pyplot as plt
from mmdet3d.datasets.pipelines.loading import LoadPointsFromFile

In [47]:
config_file = '/workspace/mmdetection3d/configs/pointpillars/hv_pointpillars_secfpn_sbn_2x16_2x_waymoD5-3d-3class.py'
cfg = mmcv.Config.fromfile(config_file)
# Set PTS voxel layer to non-deterministic to improve performance
cfg.model.pts_voxel_layer['deterministic'] = False

#this_pkg_path = rospkg.RosPack().get_path('conav_lidar_object_detection_mmdet3d')
#self.checkpoint_file = this_pkg_path + '/resources/hv_pointpillars_secfpn_sbn_2x16_2x_waymoD5-3d-3class_20200831_204144-d1a706b1.pth'

In [49]:
model = init_model(cfg, device='cuda:0')
model = model.eval().half()  # Half precision to improve performance

In [50]:
box_type_3d, box_mode_3d = get_box_type('LiDAR')

data = dict(img_metas=[[{'flip': False,
                          'pcd_horizontal_flip': False,
                          'pcd_vertical_flip': False,
                          'box_type_3d': box_type_3d,
                          'box_mode_3d': box_mode_3d,
                          'pcd_trans': np.array([0., 0., 0.]),
                          'pcd_scale_factor': 1.0,
                          'pcd_rotation': torch.tensor([[1., 0., 0.],
                                                        [-0., 1., 0.],
                                                        [0., 0., 1.]]),
                          'transformation_3d_flow': ['R', 'S', 'T']}]])

In [99]:
#file = '128c/0000000101.bin'
#file = '64c/0000000001.bin'
file = '32c/n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151603547590.pcd.bin'

In [100]:
pcd = LoadPointsFromFile('LIDAR', 4, 4)({'pts_filename':file})
pcd_numpy = pcd['points'].tensor.numpy()

In [57]:
def inference_pipeline(pcd_numpy):
    start = time.time()
    points_array = np.empty((pcd_numpy.shape[0], 5), dtype='float32')
    count = 0

    for p in pcd_numpy:
        points_array[count, 0:4] = p  # x, y, z, intensity
        count += 1


    points_array = torch.tensor(points_array).to(device='cuda')
    # Intensity in the message is 0 to 100, but maybe the model uses 0 to 1?
    points_array[:, 3] = points_array[:, 3] / 100.
    points_array[:, 4] = 0.  # 5th element is just 0
    data['points'] = [[points_array]]
    preprocessing_time = time.time() - start
    start = time.time()
    # Inference with automatic mixed precision
    with torch.cuda.amp.autocast():
        result = model(return_loss=False, rescale=False, **data)
    inference_time = time.time() - start
    return preprocessing_time, inference_time

In [102]:
warmup_runs = 50
nruns = 100

for i in range(warmup_runs):
    _,_ = inference_pipeline(pcd_numpy)
preprocessing_time, inference_time = 0,0   
for i in range(nruns):
    pre_t, inf_t = inference_pipeline(pcd_numpy)
    preprocessing_time += pre_t
    inference_time += inf_t

In [93]:
#128C
#Memory: 3383MiB
print(preprocessing_time/nruns, inference_time/nruns)
print(pcd_numpy.shape[0])

0.09483138322830201 0.04705915689468384
262144


In [98]:
#64C
#Memory: 3383MiB
print(preprocessing_time/nruns, inference_time/nruns)
print(pcd_numpy.shape[0])

0.04262580156326294 0.04689863920211792
120862


In [103]:
#32C
#Memory: 3383MiB
print(preprocessing_time/nruns, inference_time/nruns)
print(pcd_numpy.shape[0])

0.015692014694213868 0.04511390686035156
43440
